In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
import asyncio
load_dotenv(override=True)

True

In [30]:
#Tolls Agent - Capturar a Baseline para parâmetros de desempenho
# Baseline - Entres as baterias de teste, representar a melhor performance

with open("recursos/baterias-de-teste.csv", "r", encoding="utf-8") as f:
    bateriasDeTeste = f.read()

instrucoes_baseline = """
    Você é um especialista em análise de desempenho de software.
    Sua tarefa é analisar as baterias de teste e identificar a melhor performance.
    deve definir como baseline o resultado final de uma das baterias de testes de desempenho já realizadas. 
    Baseline é uma bateria que obteve o melhor resultado, seguindo os seguintes critérios: latência mais baixa, 
    taxa de erro mais baixo e vazão mais alta. Você deve apontar com base nesta relação de csv de baterias de testes, 
    com data da execução, vazão, latência e Taxa de Erro, 
    Você não precisa ser detalhista, apenas apontar a melhor bateria de teste de desempenho.
    Deve-se retorna a melhor bateria nos critérios apontado acima, apontando no seguinte formato json: 
    {
        "data": "data de execução da bateria", 
        "vazao": "vazão alcançada na bateria", 
        "latencia": "A latência alcançada na bateria", 
        "erro": "taxa de erro alcançado na bateria"
    }
"""

baseline_agent = Agent(
        name="Seletor de Baseline",
        instructions=instrucoes_baseline,
        model="gpt-5-mini",
)

baseline_tool = baseline_agent.as_tool(tool_name="baseline_agent", tool_description="Selecionar a melhor bateria de teste de desempenho")
baseline_tool


FunctionTool(name='baseline_agent', description='Selecionar a melhor bateria de teste de desempenho', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'baseline_agent_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001E753D4C180>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [31]:
#Tolls Agent - Capturar o melhor resultado alcançado em uma bateria de teste

with open("recursos/resultado-bateria1.csv", "r", encoding="utf-8") as f:
    bateriaEmanalise = f.read()

instrucoes_resultado_bateria = """
    Você é um especialista em análise de desempenho de software.
    Sua tarefa é analisar o resultado de uma bateria de teste e identificar o melhor resultado alcançado.
    O melhor resultado deve obedecer aos seguintes critérios: 
        latência não deve ser maior que 400ms, 
        taxa de erro não deve ser maior que 0.3% e 
        vazão deve ser a maior possível. 
    Você deve analisar os dados da bateria de teste no aquivo csv fornecido no com as seguintes colunas, 
    com data da execução, vazão, latência e Taxa de Erro, 
    Você não precisa ser detalhista, apenas apontar a melhor resultado alcançado.
    Deve-se retorna a melhor bateria nos critérios apontado acima, apontando no seguinte formato json: 
    {
        "data": "Deve ser a data atual", 
        "vazao": "vazão alcançada na bateria", 
        "latencia": "A latência alcançada na bateria", 
        "erro": "taxa de erro alcançado na bateria"
    }
"""


resultado_bateria_agent = Agent(
        name="Capturar resultado",
        instructions=instrucoes_resultado_bateria,
        model="gpt-5-mini",
)

resultado_bateria_tool = resultado_bateria_agent.as_tool(tool_name="resultado_bateria_agent", tool_description="Selecionar a melhor bateria de teste de desempenho")
resultado_bateria_tool



FunctionTool(name='resultado_bateria_agent', description='Selecionar a melhor bateria de teste de desempenho', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'resultado_bateria_agent_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001E753D4CE00>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [25]:
#Handoff Agent - Avaliar se o resultado da bateria de teste é melhor que a baseline
instrucoes_comparar = """
    Você é um especialista em análise de desempenho de software.
    Sua tarefa é comparar o resultado da bateria de teste com a baseline.
    você receberá do baseline_agent a baseline com os dados de data,vazao,latencia e erro.
    você receberá do resultado_bateria_agent o resultado da bateria de teste com os dados de data,vazao,latencia e erro.
    você deve avaliar se o resultado da bateria de teste é melhor que a baseline, seguindo os seguintes critérios: latência mais baixa, 
    taxa de erro mais baixo e vazão mais alta.
    Deve-se retorna a melhor bateria nos critérios apontado acima, apontando no seguinte formato json: 
    {
        "avaliacao": "Avaliação se o resultado da bateria de teste é melhor que a baseline",
        "baseline": {
            "data": "data de execução da bateria", 
            "vazao": "vazão alcançada na bateria", 
            "latencia": "A latência alcançada na bateria", 
            "erro": "taxa de erro alcançado na bateria"
        },
        "resultado_bateria": {
            "data": "data de execução da bateria", 
            "vazao": "vazão alcançada na bateria", 
            "latencia": "A latência alcançada na bateria", 
            "erro": "taxa de erro alcançado na bateria"
        }
    }
"""

comparar_agent = Agent(
    name="Comparar resultado",
    instructions=instrucoes_comparar,
    model="gpt-5-mini",
    handoff_description="Comparar resultado da bateria de teste com a baseline")


In [32]:
#Orchestrator Agent - Orquestrador para comparar o resultado da bateria de teste com as baterias já realizadas
tools = [baseline_tool, resultado_bateria_tool]
handoff_tools = [comparar_agent]


instrucoes_orchestrator = """
    Você é um orquestrador para realizar o trabalho de comparar o resultado da bateria de teste com as baterias já realizadas.
    você deverá seguir os seguintes passos:
    1 - Repassar a bateriaEmanalise para o resultado_bateria_agent
    2 - Receber o resultado da bateria de teste do resultado_bateria_agent
    3 - Receber a baseline da baseline_agent
    4 - Repassar a baseline para o comparar_agent
    5 - realizar o handoff para o comparar_agent para comparar o resultado da bateria de teste com a baseline
    6 - Retornar o resultado da comparação para o resultado_bateria_agent
"""

orchestrator = Agent(
    name="Orchestrator",
    instructions=instrucoes_orchestrator,
    tools=tools,
    handoffs=handoff_tools,
    model="gpt-5-mini")

message = f"""
    realize a comparação do resultado da bateria de teste com a baseline
    a bateria de teste é: {bateriaEmanalise}
    a baseline deve ser selecionada entre as baterias: {bateriasDeTeste}
"""


with trace("Analisar desempenho"):
    result = await Runner.run(orchestrator, message)

result

RunResult(input='\n    realize a comparação do resultado da bateria de teste com a baseline\n    a bateria de teste é: 1,10.2,120,0.0\n1,11.5,115,0.1\n1,12.8,110,0.1\n10,105.3,180,0.1\n10,112.7,175,0.2\n10,115.8,170,0.2\n20,180.5,250,0.3\n20,188.2,245,0.4\n20,192.5,240,0.4\n25,195.8,320,0.3\n25,425.3,315,0.4\n25,198.7,325,0.4\n35,415.6,420,0.2\n35,202.4,415,0.3\n35,418.9,418,0.3\n    a baseline deve ser selecionada entre as baterias: 2025-11-14,410,300,0.0\n2025-11-22,400,300,0.0\n2025-11-28,380,360,0.2\n2025-11-30,381,360,0.2\n2025-12-05,368,315,0.2\n2025-12-09,384,315,0.4\n2025-12-17,370,315,0.4\n2026-01-04,398,315,0.4\n2026-01-08,417,310,0.3\n2026-01-14,425,305,0.4\n2026-01-18,403,315,0.5\n2026-01-20,395,315,0.5\n2026-01-26,399,315,0.5\n', new_items=[ReasoningItem(agent=Agent(name='Orchestrator', handoff_description=None, tools=[FunctionTool(name='baseline_agent', description='Selecionar a melhor bateria de teste de desempenho', params_json_schema={'properties': {'input': {'title': 